<a id='intro'></a>
*To [Table of Contents](#toc)*
# Image Super Resolution
## Applied Machine Learning Systems 2
### University College London
#### Department of Electronic and Electrical Engineering
**Student Number:** $20167036$

<img src="./imgs/squirrels.png" alt="Drawing" style="height: 400px;"/>

This project focuses on generating *Super Resolution* (SR) images from *Low Resolution* (LR) ones using deep learning techniques. <br>
More specifically, the project aims at developing and evaluating deep learning models of various architectures for solving the [NTIRE2017 challenge](https://data.vision.ee.ethz.ch/cvl/ntire17/#challenge).

The datasets used for training and evaluating the models may be accessed from the [DIV2K Dataset Website](https://data.vision.ee.ethz.ch/cvl/DIV2K/) 

<a id='toc'></a>
## Table of Contents:
0. [Introduction](#intro)
1. [Loading Data](#load) 
2. [Construct Models](#construct) 
3. [Model Training](#training)
4. [Performance Evaluation](#evaluate)
5. [Credits](#credits)



---
Things to try: 
---
---
1. Dong et al. 5-1-3-3-3-3-1-9, PReLUs **Note!** PReLUs in middle, mapping layer (?) ✅
---
2. Dong et al. 5-1-3-1-9, PReLUs (note, no PReLUs in middle, mapping layer, assume 1 x (3 x 3) layer of depth 4 x 12 <br> This can't be right to be fair - the middle layers are called non-linear mapping and it's explicitly stated that he's trying to add non-linearity so must be activating ❌
---
3. Ablation 1. 5-1-3-3-3-3-1-4, PReLUs, i.e. non-overlapping deconvolutional layer scale = stride = kernel <br> **Note!** I've gotta specifically code the patch sizes for the `x3` case, make divisible by 3. Maybe just floor the 64 / scale operation, so it would be: <br>
Training patch size<br>
x4: 64 / 4 = 16 <br>
x3: 64 / 3 = 21 <br>
x3: 64 / 2 = 32   <br> 
✅ Yielded worse performance, -1 db and -0.05 SSIM
---
4. Ablation 2. 4-1-3-3-3-3-1-4, PReLUs, i.e. have kernel size match patch size in input layers<br>
✅ Much worse, grayscale outputs
---
5. Ablation 3. 3-1-3-3-1-4, PReLUs, i.e. start shrinking mapping layer, middle section
---
6. Tuning batch size
---
7. PReLU -> ReLU (?)
---
8. Weight initialisation (?)
✅ Has absolutely everything to say!
---
9. Bias initialisation (?)
---
10. L1 Loss !!!
✅ Worse
---
11. Skip connection (?) Global (?)
---

<a id='load'></a>
*Back to [Table of Contents](#toc)*
<img src="./imgs/load.png" alt="Drawing" style="height: 80px;"/>
# 1. Loading Data

Set directory (Notebooks are not hosted in base directory)

In [1]:
import os

os.chdir('..')

Configure the problem!

In [2]:
from Modules import user_interface as ui

In [3]:
### Print a welcome message
print('='*74)
greeting = 'D I V 2 K   -   S U P E R   R E S O L U T I O N'
print(greeting.center(74))
print('='*74)

# Allow user to select problem track and scaling factor
track  = ui.selection_menu('Choose track:', {'1':'Bicubic', 
                                             '2':'Unknown'})

scale = ui.selection_menu('Choose scaling factor:', {'1' : 'Factor x2',
                                                     '2' : 'Factor x3',
                                                     '3' : 'Factor x4'}) 

# Parameterise the problem, use the configuration options to generate variables that can be used to navigate directories etc.
if track is 1: 
    track = 'bicubic'
else: 
    track = 'unknown'

# Keep a numeric and string version of the scaling factor, strings for navigating directories, numeric for possibly scaling patch sizes etc
if scale is 1: 
    scale     =  2
    scale_str = 'X2'
elif scale is 2: 
    scale     =  3
    scale_str = 'X3'
else: 
    scale     =  4
    scale_str = 'X4'

# Make user aware of selection
prompt = 'You have chosen the ' + track + ' degradation problem track with ' + scale_str + ' downscaling.'
print('\n'+'='*len(prompt))
print(prompt)

             D I V 2 K   -   S U P E R   R E S O L U T I O N              


Choose track:
-------------
1. Bicubic
2. Unknown
Select by entering number and hit 'RETURN': 2


Choose scaling factor:
----------------------
1. Factor x2
2. Factor x3
3. Factor x4
Select by entering number and hit 'RETURN': 3

You have chosen the unknown degradation problem track with X4 downscaling.


### Instantiate datahandler
This project uses [idealo's datahandler from GitHub](https://idealo.github.io/image-super-resolution/), that generates patches on the fly from a directory, augments them, and limit's how flat (detail-scarce) the cropped patches can be. Further explanation on the workings of the datahandler are provided in the report and the project notebook, `project_notebook.ipynb`, where I was developing my solution and getting acquainted with using the datahandler.

In [4]:
from ISR.utils import datahandler as ISR_dh

# Define parameters for the training task, let's use 48x48 patches for the x2 downscaled images
lr_patch_size = int(128 / scale)  # Note! Patch sizes should probably be divided by scale factor
batch_size    = 4                # Hyper parameter

# We now create a datahandler for the training, and point it to the location of the LR and HR images
datahandler = ISR_dh.DataHandler(lr_dir = './Datasets/DIV2K_train_LR_' + track + '/'+ scale_str + '/',
                                 hr_dir = './Datasets/DIV2K_train_HR/',
                                 patch_size = lr_patch_size, 
                                 scale      = scale,
                                 n_validation_samples = 40)

# Generate a validation set
validation_set = datahandler.get_validation_set(batch_size = batch_size)

<a id='construct'></a>
*Back to [Table of Contents](#toc)*
<img src="./imgs/construction.png" alt="Drawing" style="height: 75px;"/>
# 2. Construct Models

I'll now define a couple of different model architectures, with varying:
- Numbers of convolutional layers
- Filter layer depths
- Kernel sizes
- Stride 

In [5]:
custom_name = str(input("Add a custom name affix to model: "))

lr_scheduler   = {'initial_value'   : 1e-4 , 
                  'decay_factor'    : 0.9  , 
                  'minimum'         : 1e-7 , 
                  'update_interval' : 10    }

adam_optimiser = {'beta1'   : 0.9   , 
                  'beta2'   : 0.999 , 
                  'epsilon' : 1e-8   }

parameters  = {# Model Structure
               'conv_layers'         : 7, 
               'conv_filters'        : [56, 16, 16, 16, 16, 16, 56],
               'conv_kernel_sizes'   : [(5,5),(1,1),(3,3),(3,3),(3,3),(3,3),(1,1)],
               'conv_strides'        : [(1,1),(1,1),(1,1),(1,1),(1,1),(1,1),(1,1)],
               'deconv_layers'       : 1, 
               'deconv_filters'      : [3],
               'deconv_kernel_sizes' : [(2*scale,2*scale)],
               'deconv_strides'      : [(scale,scale)],
               
               # Optimiser settings
               'lr_scheduler'        : lr_scheduler, 
               'adam_optimiser'      : adam_optimiser,
               
               # Affix for model name
               'name_affix'          : custom_name} # The stride of the deconvolutional layer decides the scaling

Add a custom name affix to model: _Unknown_x4


Now, I'll populate a list of models with the parameters from above

In [6]:
from Modules import model as m

model = m.SRCNN(**parameters)
model.model.summary()

Model: "Conv-7_Flt-56-16-16-16-16-16-56_Krnl-55-11-33-33-33-33-11_Strd-11-11-11-11-11-11-11-Deconv-1_Flt-3_Krnl-88_Strd-44_Unknown_x4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, None, None, 56)    4256      
_________________________________________________________________
p_re_lu (PReLU)              (None, None, None, 56)    56        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 16)    912       
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, None, None, 16)    16        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, None, None, 16)    16        
_______

<a id='training'></a>
*Back to [Table of Contents](#toc)*
<img src="./imgs/training.png" alt="Drawing" style="height: 90px;"/>
# 3. Model Training

Load models

In [7]:
if ui.yes_no_menu('Load stored model weights (y/n) ? '):
    model.loadWeights()

Load stored model weights (y/n) ? n


In [ ]:
if ui.yes_no_menu('Train model (y/n) ? '):
    
    epochs = int(input('Number of epochs to run: '))

    print('Training: \n{}'.format(model.model.name))
    model.train(total_epochs    = epochs,
                steps_per_epoch = 200,
                batch_size      = batch_size,
                datahandler     = datahandler,
                validation_set  = validation_set)

Train model (y/n) ? y
Number of epochs to run: 30
Training: 
Conv-7_Flt-56-16-16-16-16-16-56_Krnl-55-11-33-33-33-33-11_Strd-11-11-11-11-11-11-11-Deconv-1_Flt-3_Krnl-88_Strd-44_Unknown_x4
------------------------------------------------------------
            ... Initiating Training Session ...             
------------------------------------------------------------
------------------------------------------------------------
                    E P O C H   1/30                    
------------------------------------------------------------
 Runtime                   ==> 166.97866892814636s
 Learning rate             ==>     0.0001
 train_loss                ==> 0.03299858048558235
 train_psnr                ==> 14.884376525878906
 train_ssim                ==> 0.45810142159461975
 val_loss                  ==> 0.021865021961275488
 val_psnr                  ==> 17.776477813720703
 val_ssim                  ==> 0.5905104875564575
Model weights saved to: ./Models/SRCNN/Conv-7_Flt-56-1

------------------------------------------------------------
                    E P O C H   9/30                    
------------------------------------------------------------
 Runtime                   ==> 159.2905797958374s
 Learning rate             ==>     0.0001
 train_loss                ==> 0.02089763805270195
 train_psnr                ==> 16.832239151000977
 train_ssim                ==> 0.4901773929595947
 val_loss                  ==> 0.010016024045762606
 val_psnr                  ==> 22.22151756286621
 val_ssim                  ==> 0.6913560628890991
Model weights saved to: ./Models/SRCNN/Conv-7_Flt-56-16-16-16-16-16-56_Krnl-55-11-33-33-33-33-11_Strd-11-11-11-11-11-11-11-Deconv-1_Flt-3_Krnl-88_Strd-44_Unknown_x4/
************************************************************
Best model weights w/ val loss 0.010016 saved to ./Models/SRCNN/Conv-7_Flt-56-16-16-16-16-16-56_Krnl-55-11-33-33-33-33-11_Strd-11-11-11-11-11-11-11-Deconv-1_Flt-3_Krnl-88_Strd-44_Unknown_x4/
*********

------------------------------------------------------------
                    E P O C H   18/30                    
------------------------------------------------------------
 Runtime                   ==> 161.9854109287262s
 Learning rate             ==>      9e-05
 train_loss                ==> 0.018706731498241425
 train_psnr                ==> 17.576303482055664
 train_ssim                ==> 0.5368340015411377
 val_loss                  ==> 0.008601872308645397
 val_psnr                  ==> 23.255788803100586
 val_ssim                  ==> 0.7084715366363525
Model weights saved to: ./Models/SRCNN/Conv-7_Flt-56-16-16-16-16-16-56_Krnl-55-11-33-33-33-33-11_Strd-11-11-11-11-11-11-11-Deconv-1_Flt-3_Krnl-88_Strd-44_Unknown_x4/
************************************************************
Best model weights w/ val loss 0.008602 saved to ./Models/SRCNN/Conv-7_Flt-56-16-16-16-16-16-56_Krnl-55-11-33-33-33-33-11_Strd-11-11-11-11-11-11-11-Deconv-1_Flt-3_Krnl-88_Strd-44_Unknown_x4/
******

<a id='evaluate'></a>
*Back to [Table of Contents](#toc)*
<img src="./imgs/evaluate.png" alt="Drawing" style="height: 80px;"/>
# 4. Performance Evaluation

## 4.1 Learning Curves

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

Let's visualise this model's training history, the validation losses are stored in the model

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(16,6),dpi=400)

metrics = ['val_loss', 'val_psnr'  , 'val_ssim']
names   = ['MSE'     , 'PSNR (dB)' , 'SSIM'    ]

for plot,metric,name in zip(ax.ravel(),metrics,names):
    plot.plot(model.val_history[metric])
    plot.set_title(name)
    plot.set_xlabel('Epoch')

plt.legend(['Model 1: ' + str(model.model.count_params()) + ' parameters'])
plt.show()

## 4.2 Test Set

Now we can verify the performance on the test set, i.e. the 100 validation images of `DIV2K`

In [ ]:
from Modules import data_processing as dp
from Modules import metrics

In [ ]:
x_test = dp.loadImages(directory       = './Datasets/DIV2K_valid_LR_bicubic/X4/',
                       file_extension  = '.png',
                       loading_message = 'Loading [x4] Bi-Cubic Downsampled Testing Images ')
y_test = dp.loadImages(directory       = './Datasets/DIV2K_valid_HR/',
                       file_extension  = '.png',
                       loading_message = 'Loading High Resolution Testing Images \t\t')

Reshape the images into tensors using our data processing module

In [ ]:
x_test = dp.reshapeImgs(x_test)
y_test = dp.reshapeImgs(y_test)

Test set is now loaded, I'll create a `pandas` dataframe to hold my evaluation metrics.

In [ ]:
import pandas as pd

In [ ]:
results = pd.DataFrame() # Create dataframe for the evaluation scores

for sample,label in zip(x_test, y_test):
    
    y_pred  = model.model.predict(sample)               # Predict on the fullsize image
    y_pred[y_pred > 1] = 1                              # Constrain predictions to the interval [0,1]
    y_pred[y_pred < 0] = 0
    scores = metrics.evaluate(y_pred[0], label[0])      # Get a MSE, PSNR and SSIM score dictionary
    
    results = results.append(scores, ignore_index=True) # Append the score dictionary to the dataframe

Now, what's the verdict ?

In [ ]:
results.mean()

What are the worst and best predictions?

In [ ]:
print("Best prediction was on image:  \t{}.png\nWith PSNR score: \t\t{:.2f} dB".format(str(results['PSNR'].idxmax()+801).zfill(4) , results['PSNR'].max()))
print("Worst prediction was on image: \t{}.png\nWith PSNR score: \t\t{:.2f} dB".format(str(results['PSNR'].idxmin()+801).zfill(4) , results['PSNR'].min()))

## 4.3 Visual Inspection 

Now, I've taken aside a couple of $\times 4$ downscaled image and their `HR` counterparts, to perform some visual inspection of the super-resolved image quality, e.g. if we're seeing checkerboard patterns in the reconstruction etc.

In [ ]:
img_no = 0

In [ ]:
test_img  = dp.loadImages(directory       = './Datasets/Evaluate/' + track + '/'+ scale_str + '/',
                          file_extension  = '.png',
                          loading_message = 'Loading Test Images ')

In [ ]:
real_img  = dp.loadImages(directory       = './Datasets/Evaluate/HR/',
                          file_extension  = '.png',
                          loading_message = 'Loading Ground Truth Images ')

In [ ]:
test_tensor      = dp.reshapeImgs(test_img)
pred_tensor      = model.model.predict(test_tensor[img_no])
pred_tensor_list = [pred_tensor]
pred_img         = dp.reshapeTensor(pred_tensor_list)

# Constrain prediction to the [0,1] interval
pred_img[0][pred_img[0] > 1] = 1
pred_img[0][pred_img[0] < 0] = 0

# Make the original image comparable with predicted image, i.e. fit to [0,1] interval
real_img[img_no] = real_img[img_no]/255

# Evaluate MSE, PSNR and SSIM between the original and super-resolved image
image_score = metrics.evaluate(real_img[img_no],pred_img[0])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,5),sharex=True,sharey=True,dpi=400)
ax[0].imshow(real_img[img_no])
ax[0].set_title('Original')
ax[1].imshow(pred_img[0])
ax[1].set_title('Reconstruction')
plt.suptitle('PSNR: {:.2f} dB | SSIM: {:.2f}'.format(image_score['PSNR'],image_score['SSIM']))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
import numpy as np
b,h,w,ch = test_img[img_no].shape
test_img[img_no] = np.reshape(test_img[img_no],(h,w,ch))
plt.figure(figsize=(16,16),dpi=400)
plt.imshow(test_img[img_no])
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(16,16),dpi=400)
plt.imshow(pred_img[0])
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(16,16),dpi=400)
plt.imshow(real_img[img_no])
plt.xticks([])
plt.yticks([])
plt.show()

# <a id='credits'></a>
*Back to [Table of Contents](#toc)*
<img src="./imgs/credits.png" alt="Drawing" style="height: 80px;"/>
# 5. Credits


Chapter Title Icons: <a href="www.flaticon.com">Flaticon.com</a>. <br>
This notebook has been designed using resources from <a href="www.flaticon.com">Flaticon.com </a>